<a href="https://colab.research.google.com/github/KarelZe/thesis/blob/speedy-transformer/notebooks/5.0b-mb-gradient-boosting-retraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install catboost
# !pip install wandb
# !pip install gcsfs
# !pip install google-cloud-storage
# !pip install fastparquet

In [2]:
import json
import os
import pickle
from pathlib import Path

import gcsfs
import numpy as np
import pandas as pd
import wandb
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score

In [3]:
# from google.colab import auth, output
# connect to google cloud storage
# import google.auth
# auth.authenticate_user()
# credentials, _ = google.auth.default()
# fs = gcsfs.GCSFileSystem(project="thesis", token=credentials)
fs = gcsfs.GCSFileSystem(project="thesis")
# fs_prefix = "gs://"


/home/kit/stud/uloak/.local/lib/python3.8/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:


# https://github.com/KarelZe/thesis/blob/main/notebooks/
# 3.0a-mb-explanatory_data_analysis.ipynb
features_categorical = [
    ("bin_root", 8667),
    ("bin_option_type", 2),
    ("bin_issue_type", 6),
]

features_classical = [
    "TRADE_PRICE",
    "bid_ex",
    "ask_ex",
    "BEST_ASK",
    "BEST_BID",
    "price_ex_lag",
    "price_ex_lead",
    "price_all_lag",
    "price_all_lead",
    "chg_ex_lead",
    "chg_ex_lag",
    "chg_all_lead",
    "chg_all_lag",
    "prox_ex",
    "prox_best",
]

features_size = [
    "bid_ask_size_ratio_ex",
    "rel_bid_size_ex",
    "rel_ask_size_ex",
    "TRADE_SIZE",
    "bid_size_ex",
    "ask_size_ex",
    "depth_ex",
]

features_classical_size = [
    *features_classical,
    *features_size,
]


In [5]:
os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"


In [6]:
# see https://wandb.ai/fbv/thesis/runs/kwlaw02g/overview?workspace=user-karelze
run = wandb.init(project="thesis", entity="fbv")

dataset = "fbv/thesis/ise_log_standardized:v1"
artifact = run.use_artifact(dataset)
data_dir = artifact.download()

study = "fbv/thesis/xl3n4thc.optuna:v99"
artifact = run.use_artifact(study)
study_dir = artifact.download()


model = "xl3n4thc_CatBoostClassifier_default.cbm:v9"
model_name = model.split("/")[-1].split(":")[0]

artifact = run.use_artifact(model)
model_dir = artifact.download()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: karelze (fbv). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact ise_log_standardized:v1, 3813.29MB. 3 files... 
wandb:   3 of 3 files downloaded.  
Done. 0:0:4.9
/home/kit/stud/uloak/.local/lib/python3.8/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
wandb:   1 of 1 files downloaded.  
/home/kit/stud/uloak/.local/lib/python3.8/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We 

In [13]:
model = CatBoostClassifier()
model.load_model(fname=Path(model_dir, model_name))

# copy parameter, but overwrite device to gpu, if no gpu is available.
params = model.get_params()
params["task_type"] = "CPU"

model_for_refit = CatBoostClassifier(**params)

{'random_strength': 2.925840814, 'border_count': 254, 'eval_metric': 'Accuracy', 'od_wait': 100, 'iterations': 2000, 'logging_level': 'Silent', 'bagging_temperature': 0.3007743964, 'grow_policy': 'Lossguide', 'devices': '0-3', 'l2_leaf_reg': 23, 'loss_function': 'Logloss', 'task_type': 'CPU', 'depth': 11, 'learning_rate': 0.1206696523, 'random_seed': 42}


## Accuracy with retraining🎯

In [9]:
X_train = pd.read_parquet(Path(data_dir, "train_set_60.parquet"), engine="fastparquet")
y_train= X_train["buy_sell"]
X_train = X_train[features_classical_size]

X_val= pd.read_parquet(Path(data_dir, "val_set_20.parquet"), engine="fastparquet")
y_val= X_val["buy_sell"]
X_val = X_val[features_classical_size]

X_retrain = pd.concat([X_train, X_val])
del X_train, X_val

y_retrain = pd.concat([y_train, y_val])
del y_train, y_val

In [ ]:
weight = np.geomspace(0.001, 1, num=len(y_retrain))
# keep ordering of data
timestamp = np.linspace(0, 1, len(y_retrain))

# save to pool for faster memory access
retrain_pool = Pool(
            data=X_retrain,
            label=y_retrain,
            cat_features=None,
            weight=weight,
            timestamp=timestamp,
)

model_for_refit.fit(retrain_pool, verbose=False)

In [ ]:
X_test = pd.read_parquet(Path(data_dir, "test_set_20.parquet"), engine="fastparquet")
y_test = X_test["buy_sell"]
X_test = X_test[features_classical_size]

In [ ]:
acc = model.score(X_test, y_test)
print(acc)

In [ ]:
# without retraining 0.7232624886732101